# Bootcamp data science 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Spotify e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da api do Spotify para criar dashboards personalizadas que serão entregues a cada cliente baseado nas músicas, álbuns e artistas entre 2021 e 2023.

**Condições do Problema:**

1. Você recebeu os dados, em formato JSON ('Streaming_History_Audio_2021-2023.json'), com 21 colunas e mais de 10 mil linhas.
2. Seu trabalho é consumir o endpoint `https://api.spotify.com/v1/search` (API do Spotify) para obter os dados do ID e da imagem do artista.
3. Depois de obter os dados, você vai usar a base de dados dos clientes para juntar a imagem do artista em uma nova coluna. Depois que os dados estiverem limpos e as colunas que não serão utilizadas sejam apagadas
4. Uma vez que os dados estejam prontos, você vai enviar exportar essas informações para começar a criar um dashboard no Power BI.



## Importando as bibliotecas

In [1]:
import pandas as pd
import json

## Extração dos dados

### Extraindo dados da API

É importante criar um arquivo chamado .env com o CLIENT_ID e o CLIENT_SECRET

In [ ]:
import os
from dotenv import load_dotenv
import base64
from requests import post, get

In [ ]:
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

print(client_id, client_secret)

In [ ]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [ ]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [ ]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)
    return json_result

In [ ]:
token = get_token()
result = search_for_artist(token, "Taylor Swift")
# Acessar o 'id' e 'images'
artist_info = result['artists']['items'][0]  # Pegar o primeiro artista na lista 'items'
artist_id = artist_info['id']
# Pegar somente as URLs do campo das 'images'
artist_images = result['artists']['items'][0]['images']
image_urls = [image['url'] for image in artist_images]
image_url = image_urls[0]

In [ ]:
print("ID:", artist_id)
print("Image:", image_url)

#### Procurando os artistas

In [ ]:
df_songs = pd.read_json("Streaming_History_Audio_2021-2023.json")
artist_name_unique = df_songs['master_metadata_album_artist_name'].unique()

In [ ]:
artistsImage = []
tamanho = len(artist_name_unique)
for artist in artist_name_unique:
    result = search_for_artist(token, artist)
    artist_images = result['artists']['items'][0]['images']
    image_urls = [image['url'] for image in artist_images]
    artistsImage.append({
        "artist": artist,
        "images": image_urls
    })
    tamanho -= 1
    print(f"Artist: {artist}, faltam {tamanho}")

df_artists_image = pd.DataFrame(artistsImage)
df_artists_image.to_csv(encoding='latin-1')

file_path = 'artistImage.json'

with open(file_path, 'w') as json_file:
    json.dump(artistsImage, json_file)

### Extraindo dos dados das músicas

In [4]:
df_songs = pd.read_csv("my_spotify_songs.csv", encoding='latin-1')
df_songs

,Unnamed: 0,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,episode_name,episode_show_name,reason_start,reason_end
0,0,2021-06-19T18:38:41Z,228784,pra você guardei o amor,ANAVITÓRIA,N,NaN,NaN,playbtn,trackdone
1,1,2021-06-19T19:10:31Z,198835,Apê,Melim,Melim,NaN,NaN,trackdone,trackdone
2,2,2021-06-19T19:13:16Z,164257,Mapa Astral,João Guilherme,Mapa Astral,NaN,NaN,trackdone,trackdone
3,3,2021-06-19T19:16:57Z,220566,"Ai, Amor",ANAVITÓRIA,O Tempo É Agora,NaN,NaN,trackdone,trackdone
4,4,2021-06-19T19:20:14Z,166512,Peça Felicidade,Melim,Melim,NaN,NaN,trackdone,trackdone
...,...,...,...,...,...,...,...,...,...,...
12871,12871,2023-08-24T21:40:16Z,178147,good 4 u,Olivia Rodrigo,SOUR,NaN,NaN,trackdone,trackdone
12872,12872,2023-08-24T23:48:42Z,30044,One Last Time,Ariana Grande,My Everything,NaN,NaN,trackdone,unexpected-exit
12873,12873,2023-08-24T23:52:50Z,211740,Blindside,James Arthur,Blindside,NaN,NaN,remote,trackdone
12874,12874,2023-08-24T23:55:33Z,162990,past life,elijah woods,past life,NaN,NaN,trackdone,endplay


## Transformando os dados

### My songs dataset

In [8]:
df_songs.drop(columns='Unnamed: 0', inplace=True)
df_songs

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,episode_name,episode_show_name,reason_start,reason_end
0,2021-06-19T18:38:41Z,228784,pra você guardei o amor,ANAVITÓRIA,N,NaN,NaN,playbtn,trackdone
1,2021-06-19T19:10:31Z,198835,Apê,Melim,Melim,NaN,NaN,trackdone,trackdone
2,2021-06-19T19:13:16Z,164257,Mapa Astral,João Guilherme,Mapa Astral,NaN,NaN,trackdone,trackdone
3,2021-06-19T19:16:57Z,220566,"Ai, Amor",ANAVITÓRIA,O Tempo É Agora,NaN,NaN,trackdone,trackdone
4,2021-06-19T19:20:14Z,166512,Peça Felicidade,Melim,Melim,NaN,NaN,trackdone,trackdone
...,...,...,...,...,...,...,...,...,...
12871,2023-08-24T21:40:16Z,178147,good 4 u,Olivia Rodrigo,SOUR,NaN,NaN,trackdone,trackdone
12872,2023-08-24T23:48:42Z,30044,One Last Time,Ariana Grande,My Everything,NaN,NaN,trackdone,unexpected-exit
12873,2023-08-24T23:52:50Z,211740,Blindside,James Arthur,Blindside,NaN,NaN,remote,trackdone
12874,2023-08-24T23:55:33Z,162990,past life,elijah woods,past life,NaN,NaN,trackdone,endplay


In [9]:
df_songs['episode_name'].fillna('null', inplace=True)
df_songs['episode_show_name'].fillna('null', inplace=True)

In [10]:
df_songs

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,episode_name,episode_show_name,reason_start,reason_end
0,2021-06-19T18:38:41Z,228784,pra você guardei o amor,ANAVITÓRIA,N,null,null,playbtn,trackdone
1,2021-06-19T19:10:31Z,198835,Apê,Melim,Melim,null,null,trackdone,trackdone
2,2021-06-19T19:13:16Z,164257,Mapa Astral,João Guilherme,Mapa Astral,null,null,trackdone,trackdone
3,2021-06-19T19:16:57Z,220566,"Ai, Amor",ANAVITÓRIA,O Tempo É Agora,null,null,trackdone,trackdone
4,2021-06-19T19:20:14Z,166512,Peça Felicidade,Melim,Melim,null,null,trackdone,trackdone
...,...,...,...,...,...,...,...,...,...
12871,2023-08-24T21:40:16Z,178147,good 4 u,Olivia Rodrigo,SOUR,null,null,trackdone,trackdone
12872,2023-08-24T23:48:42Z,30044,One Last Time,Ariana Grande,My Everything,null,null,trackdone,unexpected-exit
12873,2023-08-24T23:52:50Z,211740,Blindside,James Arthur,Blindside,null,null,remote,trackdone
12874,2023-08-24T23:55:33Z,162990,past life,elijah woods,past life,null,null,trackdone,endplay


### Image datatset

In [12]:
df_artist_image = pd.read_json('artistImage.json')
df_artist_image

,artist,images
0,ANAVITÓRIA,[https://i.scdn.co/image/ab6761610000e5eb7ff4c...
1,Melim,[https://i.scdn.co/image/ab6761610000e5eb0efe7...
2,João Guilherme,[https://i.scdn.co/image/ab6761610000e5eb21193...
3,PEU,[https://i.scdn.co/image/ab6761610000e5eb730e7...
4,OUTROEU,[https://i.scdn.co/image/ab6761610000e5eb6ae70...
...,...,...
1002,LIT killah,[https://i.scdn.co/image/ab6761610000e5eb77d9c...
1003,Jorge Blanco,[https://i.scdn.co/image/ab6761610000e5eb73aa2...
1004,Lizzo,[https://i.scdn.co/image/ab6761610000e5eb0d66b...
1005,Tame Impala,[https://i.scdn.co/image/ab6761610000e5eb90357...


In [13]:
# Determine the maximum number of elements in any list
max_elements = df_artist_image['images'].apply(len).max()

# Create new columns based on the maximum number of elements
for i in range(max_elements):
    df_artist_image[f'image{i+1}'] = df_artist_image['images'].apply(lambda x: x[i] if i < len(x) else None)

# Drop the original 'column_with_lists' column
df_artist_image.drop(columns=['images', 'image2', 'image3', 'image4'], inplace=True)

In [14]:
df_artist_image = df_artist_image.rename(columns={'image1': 'linkImage'})
df_artist_image = df_artist_image.rename(columns={'artist': 'master_metadata_album_artist_name'})

### Criando dataframe com imagem + artista

In [15]:
df_all = pd.merge(df_songs, df_artist_image, on='master_metadata_album_artist_name', how='left')

In [16]:
df_all

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,episode_name,episode_show_name,reason_start,reason_end,linkImage
0,2021-06-19T18:38:41Z,228784,pra você guardei o amor,ANAVITÓRIA,N,null,null,playbtn,trackdone,https://i.scdn.co/image/ab6761610000e5eb7ff4c2...
1,2021-06-19T19:10:31Z,198835,Apê,Melim,Melim,null,null,trackdone,trackdone,https://i.scdn.co/image/ab6761610000e5eb0efe7e...
2,2021-06-19T19:13:16Z,164257,Mapa Astral,João Guilherme,Mapa Astral,null,null,trackdone,trackdone,https://i.scdn.co/image/ab6761610000e5eb21193d...
3,2021-06-19T19:16:57Z,220566,"Ai, Amor",ANAVITÓRIA,O Tempo É Agora,null,null,trackdone,trackdone,https://i.scdn.co/image/ab6761610000e5eb7ff4c2...
4,2021-06-19T19:20:14Z,166512,Peça Felicidade,Melim,Melim,null,null,trackdone,trackdone,https://i.scdn.co/image/ab6761610000e5eb0efe7e...
...,...,...,...,...,...,...,...,...,...,...
12871,2023-08-24T21:40:16Z,178147,good 4 u,Olivia Rodrigo,SOUR,null,null,trackdone,trackdone,https://i.scdn.co/image/ab6761610000e5ebe03a98...
12872,2023-08-24T23:48:42Z,30044,One Last Time,Ariana Grande,My Everything,null,null,trackdone,unexpected-exit,https://i.scdn.co/image/ab6761610000e5ebcdce76...
12873,2023-08-24T23:52:50Z,211740,Blindside,James Arthur,Blindside,null,null,remote,trackdone,https://i.scdn.co/image/ab6761610000e5eb4a88c3...
12874,2023-08-24T23:55:33Z,162990,past life,elijah woods,past life,null,null,trackdone,endplay,https://i.scdn.co/image/ab6761610000e5ebbe9d2d...


### Apagando colunas que não serão utilizadas

In [17]:
df_all = df_all.rename(columns={'master_metadata_track_name': 'track_name'})
df_all = df_all.rename(columns={'master_metadata_album_artist_name': 'artist_name'})
df_all = df_all.rename(columns={'master_metadata_album_album_name': 'album_name'})
df_all = df_all.rename(columns={'linkImage': 'link_image'})

## Carregando os dados (exportando)

In [18]:
df_all.to_csv('df_songs.csv')